In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_table("/content/drive/My Drive/IDVG_AllVoff.txt") 
	# print(data)
table_data = np.asarray(data)

In [0]:
new_table = np.zeros((16441, 3))

for vo in range(401):
	for i in range(41):
		new_table[41*vo + i][0], new_table[41*vo +i][1] = -4.00 + vo*0.01, (-4.0 + i*0.1) 
		new_table[41*vo + i][2] = table_data[i][vo + 1]

	# print(new_table)
new_table[:,1] = np.around(new_table[:,1], decimals=1)
df = pd.DataFrame(new_table)
	# print(df)
df.rename(columns={0: 'voff'}, inplace=True)
df.rename(columns={1: 'vg'}, inplace=True)
df.rename(columns={2: 'ids'}, inplace=True)

In [0]:
# df

In [0]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

In [0]:
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.2.0-rc2


In [0]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [0]:
dataset = df.copy()
dataset.tail()

,voff,vg,ids
16436,0.0,-0.4,0.000454
16437,0.0,-0.3,0.005877
16438,0.0,-0.2,0.062972
16439,0.0,-0.1,0.377698
16440,0.0,0.0,1.139400


In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
# train_dataset
# test_dataset

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("voff")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
vg,13153.0,-2.007078,1.182340,-4.000000e+00,-3.000000e+00,-2.000000,-1.0000,0.0000
ids,13153.0,13.143457,19.055539,5.022900e-11,5.085140e-11,0.951051,22.3692,82.7905


In [0]:
train_labels = train_dataset.pop('voff')
test_labels = test_dataset.pop('voff')

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'accoracy'])
  return model

In [0]:
model = build_model()

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 8,577
Trainable params: 8,577
Non-trainable params: 0
_________________________________________________________________


In [0]:
example_batch = train_dataset[:10]
example_result =np.around(model.predict(example_batch), decimals=2)
example_result

array([[ 0.1 ],
       [ 0.13],
       [-2.36],
       [ 0.05],
       [-2.6 ],
       [ 0.13],
       [ 0.11],
       [ 0.04],
       [ 0.17],
       [ 0.04]], dtype=float32)

In [0]:
EPOCHS = 1000

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, loss:0.5373,  mae:0.5449,  mse:0.5374,  val_loss:0.3443,  val_mae:0.3827,  val_mse:0.3433,  
....................................................................................................
Epoch: 100, loss:0.2455,  mae:0.2862,  mse:0.2456,  val_loss:0.2531,  val_mae:0.3189,  val_mse:0.2514,  
....................................................................................................
Epoch: 200, loss:0.2367,  mae:0.2773,  mse:0.2368,  val_loss:0.2415,  val_mae:0.2761,  val_mse:0.2400,  
....................................................................................................
Epoch: 300, loss:0.2318,  mae:0.2726,  mse:0.2318,  val_loss:0.2447,  val_mae:0.2969,  val_mse:0.2417,  
....................................................................................................
Epoch: 400, loss:0.2279,  mae:0.2706,  mse:0.2279,  val_loss:0.2347,  val_mae:0.2631,  val_mse:0.2331,  
........................................................................

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
995,0.223281,0.267908,0.223312,0.231410,0.279055,0.230427,995
996,0.225630,0.268698,0.225678,0.218473,0.247432,0.216243,996
997,0.222071,0.266760,0.222088,0.224276,0.261799,0.222740,997
998,0.222641,0.266861,0.222653,0.286674,0.288278,0.281275,998
999,0.221775,0.266437,0.221720,0.229845,0.260260,0.226464,999


In [0]:
test_predictions = model.predict(test_dataset).flatten()

# a = plt.axes(aspect='equal')
# plt.scatter(test_labels, test_predictions)
# plt.xlabel('True Values voff')
# plt.ylabel('Predictions voff')
# lims = [0, 50]
# plt.xlim(lims)
# plt.ylim(lims)
# _ = plt.plot(lims, lims)


In [0]:
# error = test_predictions - test_labels
# plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error voff")
# _ = plt.ylabel("Count")

In [0]:
test_predictions


array([-4.0674334 , -4.0456743 , -4.0334487 , ..., -0.9640277 ,
       -0.4965704 , -0.03533971], dtype=float32)

In [0]:
x = [-3.6]
y = [7.131340]
df = pd.DataFrame(x)
df['ids'] = y

# voff : -4.00
df.rename(columns={0: 'vg'}, inplace=True)
df
test_voff = model.predict(df).flatten()
test_voff

array([-4.011836], dtype=float32)